# Import the libraries and features

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LogisticRegression


features = np.load('features.npy')
labels = np.load('labels.npy')

print(features.shape)
print(labels.shape)


(199991, 20)
(199991,)


# Todo

implement a linear classifier on our features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=0)
X_train.shape, y_train.shape
X_test.shape, y_test.shape

#clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
#print("SVM linear :", clf.score(X_test, y_test))

clf = svm.SVC(kernel='rbf', C=1).fit(X_train, y_train)
print("SVM rbf :", clf.score(X_test, y_test))

C = 0.01

clf = LogisticRegression(C=C, solver='lbfgs', multi_class='multinomial')
clf = LogisticRegression(C=C, penalty='l2')
clf.fit(X_train, y_train)
print("Logistic : ", clf.score(X_test, y_test))

from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20, 5), random_state=1)
clf.fit(X_train, y_train)
print("Neural : ", clf.score(X_test, y_test))

# Cross Validation


In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4)
kf.get_n_splits(features)
k = 0;
for train_index, test_index in kf.split(X):
    print("fold = ", k)
    k+=1
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = LogisticRegression(C=C, solver='lbfgs', multi_class='multinomial')
    clf = LogisticRegression(C=C, penalty='l2')
    clf.fit(X_train, y_train)
    print("Logistic : ", clf.score(X_test, y_test))
